In [1]:
import json
from pathlib import Path

annotation_data_path = Path("/local/scratch/carlyn.1/datasets/ap-10k/annotations/ap10k-test-split1.json")

with open(annotation_data_path, 'r') as f:
    ann_data = json.load(f)

ann_data.keys()

dict_keys(['info', 'licenses', 'images', 'annotations', 'categories'])

In [9]:
for cat_info in ann_data['categories']:
    print(cat_info['id'], cat_info['name'])
    
# 33 giraffe

1 antelope
2 argali sheep
3 bison
4 buffalo
5 cow
6 sheep
7 arctic fox
8 dog
9 fox
10 wolf
11 beaver
12 alouatta
13 monkey
14 noisy night monkey
15 spider monkey
16 uakari
17 deer
18 moose
19 hamster
20 elephant
21 horse
22 zebra
23 bobcat
24 cat
25 cheetah
26 jaguar
27 king cheetah
28 leopard
29 lion
30 panther
31 snow leopard
32 tiger
33 giraffe
34 hippo
35 chimpanzee
36 gorilla
37 orangutan
38 rabbit
39 skunk
40 mouse
41 rat
42 otter
43 weasel
44 raccoon
45 rhino
46 marmot
47 squirrel
48 pig
49 mole
50 black bear
51 brown bear
52 panda
53 polar bear
54 bat


In [6]:
for key, value in ann_data['annotations'][0].items():
    print(key, value)

id 114
image_id 102
category_id 1
bbox [424, 205, 552, 456]
area 251712
iscrowd 0
num_keypoints 10
keypoints [885, 355, 2, 810, 361, 2, 843, 412, 2, 864, 529, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 729, 571, 2, 822, 604, 2, 678, 619, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 590, 473, 2, 456, 622, 2, 549, 631, 2]


In [16]:
import os
# Count number of data files (10015 files)
total = 0
for _, _, files in os.walk(Path("/local/scratch/carlyn.1/datasets/ap-10k/data")):
    total += len(files)

total

10015